In [1]:
! pip install wget
#! apt-get install htop

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=d0bf26a288ea785f6cbef9e0e644c6328ca30474791ec0c287bcfb33c32dc99f
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
Suggested packages:
  strace
The following NEW packages will be installed:
  htop
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 80.0 kB of archives.
After this operation, 221 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 htop amd64 2.1.0-3 [80.0 kB]
Fetched 80.0 kB in 0s (185 kB/s)
Selecting previously unselected package htop.
(Reading database ... 144487 files and directories currently installed.)
Preparing

In [2]:
! mkdir Kolkata_colab

In [3]:
"""
This program downloads / scraps Apple maps for free.
OPEN SOURCED UNDER GPL-V3.0.
Author : Jimut Bahan Pal | jimutbahanpal@yahoo.com
"""

import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import os
import cv2
import wget
import imghdr
import shutil
import numpy as np
import requests
from time import time as timer
from multiprocessing.pool import ThreadPool
import time
import math
from os import listdir
from os.path import isfile, join
from tqdm import tqdm

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.8; rv:21.0) Gecko/20100101 Firefox/21.0'
}

LOCK_VAR = 0
UNLOCK_VAR = 0
LOCKING_LIMIT = 50 # MAX NO OF THREADS


class api:
    
    def __init__(self,ac_key,min_lat_deg,max_lat_deg,min_lon_deg,max_lon_deg,zoom=19,verbose=False,threads_=50):
        global LOCKING_LIMIT
        self.ac_key = ac_key
        self.min_lat_deg = min_lat_deg
        self.max_lat_deg = max_lat_deg
        self.min_lon_deg = min_lon_deg
        self.max_lon_deg = max_lon_deg
        self.zoom = zoom
        self.verbose = verbose
        LOCKING_LIMIT = threads_
        print(self.ac_key,self.min_lat_deg,self.max_lat_deg,self.min_lon_deg,self.max_lon_deg,self.zoom,self.verbose,LOCKING_LIMIT)

    def ret_xy_tiles(self,lat_deg,lon_deg):
        # changes for 0.0005
        # This function returns the tilex and tiley in tuple
        # Takes latitude, longitude and zoom_level
        n = 2**self.zoom
        xtile = n * ((lon_deg + 180) / 360)
        lat_rad = lat_deg * math.pi / 180.0
        ytile = n * (1 - (math.log(math.tan(lat_rad) + 1/math.cos(lat_rad)) / math.pi)) / 2
        return int(xtile),int(ytile)

    def ret_lat_lon(self,x_tyle,y_tyle):
        # This function returns the lat, lon as a tuple
        # Takes x_tyle, y_tyle and zoom_level
        n = 2**self.zoom
        lon_deg = int(x_tyle)/n * 360.0 - 180.0
        lat_rad = math.atan(math.asinh(math.pi * (1 - 2 * int(y_tyle)/n)))
        lat_deg = lat_rad * 180.0 / math.pi
        return lat_deg, lon_deg
    
    def make_url(self,lat_deg,lon_deg):
        # returns the list of urls when lat, lon, zoom and accessKey is provided
        x_tyle,y_tyle = self.ret_xy_tiles(lat_deg,lon_deg)
        return [x_tyle,y_tyle]
    
    def get_img(self,url_str):
        # to get the images from the url provided and save it
        global headers, LOCK_VAR, UNLOCK_VAR, LOCKING_LIMIT
        if self.verbose == True:
            print(url_str)
        UNLOCK_VAR = UNLOCK_VAR + 1
        LOCK_VAR = 1
        if self.verbose == True:
            print("UNLOCK VAR : ",UNLOCK_VAR)
        if UNLOCK_VAR >= LOCKING_LIMIT:
            LOCK_VAR = 0
            UNLOCK_VAR = 0
            if self.verbose == True:
                print("-------- UNLOCKING")
        x_tyle = url_str[0]
        y_tyle = url_str[1]
        file_name = str(x_tyle)+"_"+str(y_tyle)+".jpeg"
        try:
            if open(str(file_name),'r') == True:
                if self.verbose == True:
                    print(file_name,"file is present")
                pass
        except:
            try:
                #https://sat-cdn3.apple-mapkit.com/tile?style=7&size=1&scale=1&z=19&x=371320&y=219633&v=7072&accessKey=1597844285_2623996721767830360_%2F_2JSDgZLx9BNvymyuDPqmaVPQGKgIY6HNsMGvnWnVP74%3D
                #https://cdn3.apple-mapkit.com/ti/tile?country=IN&region=IN&style=46&size=1&x=371305&y=219636&z=19&scale=1&lang=en&v=2008184&poi=0&accessKey=1597846063_5668354640921081798_%2F_tDWzoV7s8c94X9yDuhnpOfUQadPrp82ogZuM%2BHwf50g%3D&labels=0
                #https://cdn1.apple-mapkit.com/ti/tile?country=IN&region=IN&style=46&size=1&x=371307&y=219638&z=19&scale=1&lang=en&v=2008184&poi=0&accessKey=1597846063_5668354640921081798_%2F_tDWzoV7s8c94X9yDuhnpOfUQadPrp82ogZuM%2BHwf50g%3D&labels=0
                req_url = str("https://sat-cdn"+str(1)+".apple-mapkit.com/tile?style=7&size=1&scale=1&z="+str(self.zoom)+"&x="+str(x_tyle)+"&y="+str(y_tyle)+"&v=7072"+str(self.ac_key))
                if self.verbose == True:
                    print(req_url)
                img_name = file_name.split('.')[0]
                file_name1 = str(img_name + ".jpeg")
                r = requests.get(req_url, #allow_redirects=True,
                                headers=headers)
                open(file_name1, 'wb').write(r.content)
                if imghdr.what(file_name1) is 'jpeg':
                    if self.verbose == True:
                        print(file_name1,"JPEG")
                else:
                    os.remove(file_name1)
                    if self.verbose == True:
                        print(file_name1,"NOT JPEG")
                
                # For the roads data
                req_url = str("https://cdn"+str(1)+".apple-mapkit.com/ti/tile?country=IN&region=IN&style=46&size=1&x=")+str(x_tyle)+str("&y=")+str(y_tyle)+str("&z=")+str(self.zoom)+"&scale=1&lang=en&v=2008184&poi=0"+str(self.ac_key)+"&labels=0"
                #req_url = str("https://sat-cdn"+str(1)+".apple-mapkit.com/tile?style=7&size=1&scale=1&z="+str(self.zoom)+"&x="+str(x_tyle)+"&y="+str(y_tyle)+"&v=7072"+str(self.ac_key))
                
                file_name_road = file_name.split('.')[0]+"_road.png"
                if self.verbose == True:
                    print(req_url)
                r = requests.get(req_url, #allow_redirects=True,
                                headers=headers)
                open(file_name_road, 'wb').write(r.content)
                if imghdr.what(file_name_road) is 'png':
                    if self.verbose == True:
                        print(file_name_road,"PNG")
                else:
                    os.remove(file_name_road)
                    if self.verbose == True:
                        print(file_name_road,"NOT PNG")
            except Exception as e:
                if self.verbose == True:
                    print(e)
            
            try:
                """
                req_url = str("https://sat-cdn"+str(2)+".apple-mapkit.com/tile?style=7&size=1&scale=1&z="+str(self.zoom)+"&x="+str(x_tyle)+"&y="+str(y_tyle)+"&v=4072"+str(self.ac_key))
                if self.verbose == True:
                    print(req_url)
                r = requests.get(req_url, #allow_redirects=True,
                                headers=headers)
                file_name2 = str(img_name + "_2.jpeg")
                open(file_name2, 'wb').write(r.content)
                if imghdr.what(file_name2) is 'jpeg':
                    if self.verbose == True:
                        print(file_name2,"JPEG")
                else:
                    os.remove(file_name2)
                    if self.verbose == True:
                        print(file_name2,"NOT JPEG")
                """
                # For the roads data
                req_url = str("https://cdn"+str(2)+".apple-mapkit.com/ti/tile?country=IN&region=IN&style=46&size=1&x=")+str(x_tyle)+str("&y=")+str(y_tyle)+str("&z=")+str(self.zoom)+"&scale=1&lang=en&v=2008184&poi=0"+str(self.ac_key)+"&labels=0"
                
                if self.verbose == True:
                    print(req_url)
                r = requests.get(req_url, #allow_redirects=True,
                                headers=headers)
                open(file_name_road, 'wb').write(r.content)
                if imghdr.what(file_name_road) is 'png':
                    if self.verbose == True:
                        print(file_name_road,"PNG")
                else:
                    os.remove(file_name_road)
                    if self.verbose == True:
                        print(file_name_road,"NOT PNG")
                        
            except Exception as e:
                if self.verbose == True:
                    print(e)
            
                
            try:
                """
                req_url = str("https://sat-cdn"+str(3)+".apple-mapkit.com/tile?style=7&size=1&scale=1&z="+str(self.zoom)+"&x="+str(x_tyle)+"&y="+str(y_tyle)+"&v=4072"+str(self.ac_key))
                if self.verbose == True:
                    print(req_url)
                r = requests.get(req_url, #allow_redirects=True,
                                headers=headers)
                file_name3 = str(img_name + "_3.jpeg")
                open(file_name3, 'wb').write(r.content)
                if imghdr.what(file_name3) is 'jpeg':
                    if self.verbose == True:
                        print(file_name3,"JPEG")
                else:
                    os.remove(file_name3)
                    if self.verbose == True:
                        print(file_name3,"NOT JPEG")
                """
                # For the roads data
                req_url = str("https://cdn"+str(3)+".apple-mapkit.com/ti/tile?country=IN&region=IN&style=46&size=1&x=")+str(x_tyle)+str("&y=")+str(y_tyle)+str("&z=")+str(self.zoom)+"&scale=1&lang=en&v=2008184&poi=0"+str(self.ac_key)+"&labels=0"
                #print(req_url)
                if self.verbose == True:
                    print(req_url)
                r = requests.get(req_url, #allow_redirects=True,
                                headers=headers)
                open(file_name_road, 'wb').write(r.content)
                if imghdr.what(file_name_road) is 'png':
                    if self.verbose == True:
                        print(file_name_road,"PNG")
                else:
                    os.remove(file_name_road)
                    if self.verbose == True:
                        print(file_name_road,"NOT PNG")
                        
            except Exception as e:
                if self.verbose == True:
                    print(e)
            try:
                """
                req_url = str("https://sat-cdn"+str(4)+".apple-mapkit.com/tile?style=7&size=1&scale=1&z="+str(self.zoom)+"&x="+str(x_tyle)+"&y="+str(y_tyle)+"&v=4072"+str(self.ac_key))
                if self.verbose == True:
                    print(req_url)
                r = requests.get(req_url, #allow_redirects=True,
                                headers=headers)
                file_name4 = str(img_name + "_4.jpeg")
                open(file_name4, 'wb').write(r.content)
                if imghdr.what(file_name4) is 'jpeg':
                    if self.verbose == True:
                        print(file_name4,"JPEG")
                else:
                    os.remove(file_name4)
                    if self.verbose == True:
                        print(file_name4,"NOT JPEG")
                """
                # For the roads data
                req_url = str("https://cdn"+str(4)+".apple-mapkit.com/ti/tile?country=IN&region=IN&style=46&size=1&x=")+str(x_tyle)+str("&y=")+str(y_tyle)+str("&z=")+str(self.zoom)+"&scale=1&lang=en&v=2008184&poi=0"+str(self.ac_key)+"&labels=0"
                #req_url = str("https://sat-cdn"+str(1)+".apple-mapkit.com/tile?style=7&size=1&scale=1&z="+str(self.zoom)+"&x="+str(x_tyle)+"&y="+str(y_tyle)+"&v=7072"+str(self.ac_key))
                
                
                if self.verbose == True:
                    print(req_url)
                r = requests.get(req_url, #allow_redirects=True,
                                headers=headers)
                open(file_name_road, 'wb').write(r.content)
                if imghdr.what(file_name_road) is 'png':
                    if self.verbose == True:
                        print(file_name_road,"PNG")
                else:
                    os.remove(file_name_road)
                    if self.verbose == True:
                        print(file_name_road,"NOT PNG")
                        
            except Exception as e:
                if self.verbose == True:
                    print(e)
        # delete unnecessary files

    def download(self):
        min_lat = self.min_lat_deg
        max_lat = self.max_lat_deg
        min_lon = self.min_lon_deg
        max_lon = self.max_lon_deg
        if min_lat > max_lat:
            i_val = -1
        else:
            i_val = 1
        
        if max_lon > max_lon:
            j_val = -1
        else:
            j_val = 1
        
        for i in tqdm(np.arange(float(min_lat),float(max_lat),i_val*0.0005)):
            URL_ALL = []
            for j in np.arange(float(min_lon),float(max_lon),j_val*0.0005):
                get_urls = self.make_url(i,j)
                URL_ALL.append([get_urls[0],get_urls[1]])
            if self.verbose == True:
                print("ALL URL CREATED! ...")
            global LOCK_VAR, UNLOCK_VAR, LOCKING_LIMIT
            if LOCK_VAR == 0:
                if self.verbose == True:
                    print("LOCKING")
                LOCK_VAR = 1
                UNLOCK_VAR = 0
                ThreadPool(LOCKING_LIMIT).imap_unordered(self.get_img, URL_ALL)
            # SEMAPHORE KINDA THINGIE
            while LOCK_VAR == 1:
                
                if self.verbose == True:
                    print("WAITING",end="")
                pass

In [ ]:
import sqlite3

conn = sqlite3.connect('map_db.sqlite')
cur = conn.cursor()

get_items = cur.execute(''' SELECT * FROM data where visited = ?''',(0,))

counter_i = 0
for item in get_items:
    #print(item)
    id_ = item[0]
    min_lat_deg = float(item[1])
    max_lat_deg = float(item[2])
    min_lon_deg = float(item[3])
    max_lon_deg = float(item[4])
    if counter_i %3 == 0:
      api_key = input("Give API (from &accessKey=) ::")
    counter_i += 1
    a = api(str(api_key),min_lat_deg,max_lat_deg,min_lon_deg,max_lon_deg,zoom=19,verbose=False,threads_=6)
    a.download()
    print("Done, comitting")
    cur.execute('''UPDATE data SET visited = 1 WHERE id = ?''',(id_,))
    cur.execute('''UPDATE data SET key = ? WHERE id = ?''',(api_key, id_,))
    conn.commit()
    
    


Give API (from &accessKey=) ::&accessKey=1597913196_587545248982524064_%2F_eymjhVKEOd8pTORsYbXuXZQXgq4tJbBVRc%2FOdFCS5FY%3D


  3%|▎         | 3/101 [00:00<00:04, 23.25it/s]

&accessKey=1597913196_587545248982524064_%2F_eymjhVKEOd8pTORsYbXuXZQXgq4tJbBVRc%2FOdFCS5FY%3D 22.400000000000002 22.450000000000003 88.24999999999999 88.29999999999998 19 False 6


 44%|████▎     | 44/101 [00:48<02:27,  2.59s/it]

In [ ]:
! mv *.jpeg Kolkata_colab
! mv *.png Kolkata_colab

In [55]:
! du -hs  Kolkata_colab

51M	Kolkata_colab


In [58]:
! rm -rf Kolkata_colab.zip

In [59]:
! zip -r Kolkata_colab.zip Kolkata_colab > /etc/null

In [60]:
! du -hs Kolkata_colab.zip

34M	Kolkata_colab.zip


In [61]:
! rm -rf Kolkata_colab